# Name Anonymization for privacy

using [FlairNLP](https://github.com/flairNLP/flair) to anonymize Names in the patient reviews dataset 




In [1]:
# imports
import torch
import pandas as pd
import re
from flair.data import Sentence
from flair.models import SequenceTagger

/home/sc.uni-leipzig.de/ch31qoni/venv/absa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Is CUDA available: True
CUDA version: 12.4
GPU device name: Tesla V100-PCIE-32GB


In [3]:
# Load the pre-trained German NER model
tagger = SequenceTagger.load("flair/ner-german-large")

# Function to anonymize names while keeping titles
def anonymize_text(text):
    sentence = Sentence(text)
    tagger.predict(sentence)

    for entity in sentence.get_spans('ner'):
        if entity.tag == "PER":  # Check if the entity is a person
            name_parts = entity.text.split()  # Split the detected name
            if len(name_parts) > 1:
                title = " ".join(name_parts[:-1])  # Keep the title (e.g., Frau Dr.)
                text = text.replace(entity.text, f"{title} [NAME]")
            else:
                text = text.replace(entity.text, "[NAME]")  # If no title, replace everything
    
    return text

2025-02-27 16:10:53,256 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


In [ ]:
# Load dataset
df = pd.read_csv("./data/hospitalABSA/patient_review_labels_absa.csv")


In [4]:
# List of sentence_ids to keep
selected_sentence_ids = [5100, 5112, 7264, 7474, 7479, 8385, 4961, 5004, 5005, 5006, 5007, 5008]

# Filter DataFrame
filtered_df = df[df["sentence_id"].isin(selected_sentence_ids)]

# Show the result
filtered_df

,review_id,sentenceId,raw_text,aspectTerms,aspectCategories
256,31.0,4961,"Vor allem gilt großer Dank Prof. Dr. Med. N., ...","[{'term': 'Prof. Dr. Med. N.', 'polarity': 'po...","[{'category': 'Arzt', 'polarity': 'positiv'}]"
295,35.0,5004,Frau Dr. Hohdorf nimmt sich immer Zeit und erk...,"[{'term': 'Frau Dr. Hohdorf', 'polarity': 'pos...","[{'category': 'Arzt', 'polarity': 'positiv'}]"
296,35.0,5005,"Mein Geburtstag war da schon eine Woche alt, t...","[{'term': 'Frau Dr. Hohdorf', 'polarity': 'pos...","[{'category': 'Arzt', 'polarity': 'positiv'}]"
297,35.0,5006,Danke Frau Dr. Hohdorf,"[{'term': 'Frau Dr. Hohdorf', 'polarity': 'pos...","[{'category': 'Arzt', 'polarity': 'positiv'}]"
298,35.0,5007,"Ich war kurzfristig am Freitag, den 20.10.2023...","[{'term': 'Sonografie', 'polarity': 'neutral'}...","[{'category': 'mediz. Service', 'polarity': 'n..."
299,35.0,5008,Durchgeführt wurde die Sonografie von Frau Dr....,"[{'term': 'Sonografie', 'polarity': 'neutral'}...","[{'category': 'mediz. Service', 'polarity': 'n..."
379,46.0,5100,Deshalb ein herzlichen Dankeschön an alle Mita...,"[{'term': 'Mitarbeitende', 'polarity': 'positi...","[{'category': 'Personal', 'polarity': 'positiv'}]"
380,47.0,5112,Ein herzliches Dankeschön an Frau Dr. Molterer...,"[{'term': 'Frau Dr. Molterer', 'polarity': 'po...","[{'category': 'Arzt', 'polarity': 'positiv'}, ..."
576,292.0,7264,Besonders förderlich für den Genesungsprozess ...,"[{'term': 'Kurzvisite', 'polarity': 'positiv'}...","[{'category': 'mediz. Service', 'polarity': 'p..."
590,309.0,7474,Am Mittwoch früh zur Chefvisite schilderte ich...,"[{'term': 'Chefvisite', 'polarity': 'neutral'}...","[{'category': 'mediz. Service', 'polarity': 'n..."


In [5]:
# Apply anonymization
filtered_df["raw_text"]= filtered_df["raw_text"].apply(anonymize_text)
filtered_df["raw_text"]

/tmp/ipykernel_3101645/3666607915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["raw_text"]= filtered_df["raw_text"].apply(anonymize_text)


256    Vor allem gilt großer Dank Prof. Dr. Med. [NAM...
295    Frau Dr. [NAME] nimmt sich immer Zeit und erkl...
296    Mein Geburtstag war da schon eine Woche alt, t...
297                                Danke Frau Dr. [NAME]
298    Ich war kurzfristig am Freitag, den 20.10.2023...
299    Durchgeführt wurde die Sonografie von Frau Dr....
379    Deshalb ein herzlichen Dankeschön an alle Mita...
380    Ein herzliches Dankeschön an Frau Dr. [NAME], ...
576    Besonders förderlich für den Genesungsprozess ...
590    Am Mittwoch früh zur Chefvisite schilderte ich...
595    Nach Rücksprache mit dem Professor mußten sie ...
680    Ich habe dieser  Klinik (Prof. Dr. [NAME]) vie...
Name: raw_text, dtype: object

In [ ]:
# Load dataset
patient_df = pd.read_csv("data/sentence_data/labeling/patient_review_labels_labeled.csv",sep=";")
patient_df

,reviewID,sentence,AspectCategory,AspectTerm,polarity,from,to
0,1.0,Ich bin heute hier angekommen.,Allgemein,NaN,neutral,NaN,NaN
1,1.0,Ärzte und Pflegepersonal ist in Ordnung.,Arzt,Ärzte,positiv,0.0,4.0
2,1.0,Ärzte und Pflegepersonal ist in Ordnung.,Pflegepersonal,Pflegepersonal,positiv,10.0,23.0
3,1.0,Was hier nicht stimmt ist die Sauberkeit und O...,Krankenhaus,Zimmer,negativ,56.0,61.0
4,1.0,Was hier nicht stimmt ist die Sauberkeit und O...,Krankenhaus,Sauberkeit,negativ,30.0,39.0
...,...,...,...,...,...,...,...
5170,486.0,Mein nachträglicher Dank an dieser Stelle alle...,Pflegepersonal,Pflegepersonal,positiv,74.0,87.0
5171,486.0,Die einfache Zweibettzimmerausstattung (Altgeb...,Krankenhaus,Zweibettzimmerausstattung,positiv,13.0,37.0
5172,486.0,Die einfache Zweibettzimmerausstattung (Altgeb...,Arzt,ärztlichen,positiv,165.0,174.0
5173,486.0,Die einfache Zweibettzimmerausstattung (Altgeb...,anderer Service,pflegenden Leistungen,positiv,180.0,200.0


In [7]:
# Apply anonymization
patient_df["sentence"]= patient_df["sentence"].apply(anonymize_text)


In [ ]:
# Save the anonymized dataset
patient_df.to_csv("data/sentence_data/labeling/patient_review_labels_anonym.csv", index=False, quoting=1, sep=";")

In [9]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5175 entries, 0 to 5174
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewID        5175 non-null   float64
 1   sentence        5175 non-null   object 
 2   AspectCategory  1507 non-null   object 
 3   AspectTerm      1393 non-null   object 
 4   polarity        1507 non-null   object 
 5   from            1362 non-null   float64
 6   to              1362 non-null   float64
dtypes: float64(3), object(4)
memory usage: 283.1+ KB


In [10]:
patient_df["AspectTerm"] = patient_df["AspectTerm"].apply(lambda x: anonymize_text(x) if isinstance(x, str) else x)

In [ ]:
# Save the anonymized dataset
patient_df.to_csv("./data/hospitalABSA/patient_review_labels_anonym.csv", index=False, quoting=1, sep=";")